In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder, StandardScaler # encode
# SVM
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-student-performance-from-game-play/sample_submission.csv
/kaggle/input/predict-student-performance-from-game-play/train_labels.csv
/kaggle/input/predict-student-performance-from-game-play/train.csv
/kaggle/input/predict-student-performance-from-game-play/test.csv
/kaggle/input/predict-student-performance-from-game-play/jo_wilder/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/predict-student-performance-from-game-play/jo_wilder/__init__.py


In [2]:
'''

df = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train.csv')
print(df.head())
print(df.columns)
print(df.shape)

'''

"\n\ndf = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train.csv')\nprint(df.head())\nprint(df.columns)\nprint(df.shape)\n\n"

Useful Notebook: https://www.kaggle.com/code/chiaenchen/eda-predict-student-performance-from-game-play#Conclusion




1. Drop the feature which 'null' ratio > 50%
2. Encode string value, event_name and name
3. Fill missing item in coordinates by Forward fill
4. Remove features that are not relevant to predicting game performance, hover_duration, fullscreen, hq,music

In [3]:
dtypes = {"session_id": 'int64',
          "index": np.int16,
          "elapsed_time": np.int32,
          "event_name": 'category',
          "name": 'category',
          "level": np.int8,
          "page": np.float16,
          "room_coor_x": np.float64,
          "room_coor_y": np.float64,
          "screen_coor_x": np.float64,
          "screen_coor_y": np.float64,
          "hover_duration": np.float32,
          "text": 'category',
          "fqid": 'category',
          "room_fqid": 'category',
          "text_fqid": 'category',
          "fullscreen": np.int8,
          "hq": np.int8,
          "music": np.int8,
          "level_group": 'category'
          }

In [4]:
#avoid memory error, use chunk
def process(chunk,data):
    # process data here
    #print(chunk.shape)
    
    # Remove the 'page','text_fqid,'fqid','text' column
    chunk.drop('page', axis=1, inplace=True)
    chunk.drop('text_fqid', axis=1, inplace=True)
    chunk.drop('fqid', axis=1, inplace=True)
    chunk.drop('text', axis=1, inplace=True)
    
    # Remove Unrelate value
    chunk.drop('hover_duration', axis=1, inplace=True)
    chunk.drop('fullscreen', axis=1, inplace=True)
    chunk.drop('hq', axis=1, inplace=True)
    chunk.drop('music', axis=1, inplace=True)

    # Encode the 'even_name',  variable
    encoder = LabelEncoder()
    chunk['event_name'] = encoder.fit_transform(chunk['event_name'])
    chunk['name'] = encoder.fit_transform(chunk['name'])
    chunk['room_fqid'] = encoder.fit_transform(chunk['room_fqid'])
    
    # fill missing item by previous value, 
    chunk[['room_coor_x', 'room_coor_y']].fillna(method='ffill', inplace=True)
    chunk[['screen_coor_x', 'screen_coor_y']].fillna(method='ffill', inplace=True)
    
    # scale the data
    ...

    # Add the preprocessed chunk to the list
    data.append(chunk)

In [5]:
# load in data

#avoid memory error, set nrows to 100000
chunk_size = 10000
data = []
data_generator = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train.csv', chunksize=chunk_size,dtype=dtypes,nrows = 100000)
for chunk in data_generator:
    process(chunk,data)
# Concatenate all preprocessed chunks into a single DataFrame
data = pd.concat(data, axis=0)

# Load target label
target = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train_labels.csv')


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [6]:
#print(preprocessed_data[0:100])
data.shape
target.shape
data.head(3)

,session_id,index,elapsed_time,event_name,name,level,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,room_fqid,level_group
0,20090312431273200,0,0,1,0,0,-413.991405,-159.314686,380.0,494.0,7,0-4
1,20090312431273200,1,1323,10,0,0,-413.991405,-159.314686,380.0,494.0,7,0-4
2,20090312431273200,2,831,10,0,0,-413.991405,-159.314686,380.0,494.0,7,0-4


In [7]:
data.elapsed_time = data.elapsed_time/60000 #change millsecond to mins
data.to_csv("/kaggle/working/preprocessed_data", index=False)
data.head(50)

,session_id,index,elapsed_time,event_name,name,level,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,room_fqid,level_group
0,20090312431273200,0,0.000000,1,0,0,-413.991405,-159.314686,380.0,494.0,7,0-4
1,20090312431273200,1,0.022050,10,0,0,-413.991405,-159.314686,380.0,494.0,7,0-4
2,20090312431273200,2,0.013850,10,0,0,-413.991405,-159.314686,380.0,494.0,7,0-4
3,20090312431273200,3,0.019117,10,0,0,-413.991405,-159.314686,380.0,494.0,7,0-4
4,20090312431273200,4,0.031050,10,0,0,-412.991405,-159.314686,381.0,494.0,7,0-4
5,20090312431273200,5,0.057050,10,0,0,-412.991405,-157.314686,381.0,492.0,7,0-4
6,20090312431273200,6,0.086617,10,0,0,478.485079,-199.971679,593.0,485.0,7,0-4
7,20090312431273200,7,0.103000,10,0,0,503.355128,-168.619913,609.0,453.0,7,0-4
8,20090312431273200,8,0.116900,10,0,0,510.733442,-157.720642,615.0,442.0,7,0-4
9,20090312431273200,9,0.132433,10,0,0,512.048005,-153.743631,616.0,438.0,7,0-4


The session_id of target contains the number of topics (q1-q8), which cannot be matched with the session_id of data. Because, add a new user_id which in target.(this idea refer to chiaenchen)
Use regular expressions to achieve this.

In [8]:
import re #regular expression

# split session_id by "-". -> 20090312431273200_q1: 20090312431273200 and 1
target["session_id"],target["q_id"] = target.session_id.str.split("_", expand = True)[0],target.session_id.str.split("_", expand = True)[1] # session_id is 20090312431273200
target["q_id"] = target["q_id"].apply(lambda x : re.sub("\D", "",x)) # remove all non-digit characters from the "question" column 

# to numeric
target["q_id"] = pd.to_numeric(target["q_id"])
target["session_id"] = pd.to_numeric(target["session_id"])

target.head(3)


,session_id,correct,q_id
0,20090312431273200,1,1
1,20090312433251036,0,1
2,20090312455206810,1,1


list of question; 
number of user in data; 
number of user in target; 




In [9]:
target.q_id.unique()
data.session_id.nunique()
target.session_id.nunique()
data.shape,target.shape

((100000, 12), (424116, 3))

In [10]:
target.to_csv("/kaggle/working/extended_target", index=False)

Data analysis

In [11]:
# data info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   session_id     100000 non-null  int64   
 1   index          100000 non-null  int16   
 2   elapsed_time   100000 non-null  float64 
 3   event_name     100000 non-null  int64   
 4   name           100000 non-null  int64   
 5   level          100000 non-null  int8    
 6   room_coor_x    91474 non-null   float64 
 7   room_coor_y    91474 non-null   float64 
 8   screen_coor_x  91474 non-null   float64 
 9   screen_coor_y  91474 non-null   float64 
 10  room_fqid      100000 non-null  int64   
 11  level_group    100000 non-null  category
dtypes: category(1), float64(5), int16(1), int64(4), int8(1)
memory usage: 7.2 MB


In [12]:
# user1's data
id1 = data.session_id[0]
data1 = data.loc[data.session_id == id1]
data1

,session_id,index,elapsed_time,event_name,name,level,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,room_fqid,level_group
0,20090312431273200,0,0.000000,1,0,0,-413.991405,-159.314686,380.0,494.0,7,0-4
1,20090312431273200,1,0.022050,10,0,0,-413.991405,-159.314686,380.0,494.0,7,0-4
2,20090312431273200,2,0.013850,10,0,0,-413.991405,-159.314686,380.0,494.0,7,0-4
3,20090312431273200,3,0.019117,10,0,0,-413.991405,-159.314686,380.0,494.0,7,0-4
4,20090312431273200,4,0.031050,10,0,0,-412.991405,-159.314686,381.0,494.0,7,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...
876,20090312431273200,927,21.122617,4,5,22,927.307255,-10.355929,838.0,335.0,11,13-22
877,20090312431273200,928,21.138200,3,0,22,NaN,NaN,NaN,NaN,11,13-22
878,20090312431273200,929,21.157900,2,5,22,457.523005,22.141338,443.0,316.0,11,13-22
879,20090312431273200,930,21.178467,4,5,22,224.190321,-60.268671,404.0,337.0,2,13-22


In [13]:
# user1's answer
target[target.session_id == id1]

,session_id,correct,q_id
0,20090312431273200,1,1
23562,20090312431273200,1,2
47124,20090312431273200,1,3
70686,20090312431273200,1,4
94248,20090312431273200,1,5
117810,20090312431273200,1,6
141372,20090312431273200,1,7
164934,20090312431273200,1,8
188496,20090312431273200,1,9
212058,20090312431273200,1,10


Process data

In [14]:
# group by session_id
grouped_data = data.groupby('session_id')
grouped_data.head(3)

,session_id,index,elapsed_time,event_name,name,level,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,room_fqid,level_group
0,20090312431273200,0,0.000000,1,0,0,-413.991405,-159.314686,380.0,494.0,7,0-4
1,20090312431273200,1,0.022050,10,0,0,-413.991405,-159.314686,380.0,494.0,7,0-4
2,20090312431273200,2,0.013850,10,0,0,-413.991405,-159.314686,380.0,494.0,7,0-4
881,20090312433251036,0,0.000000,1,0,0,-394.991405,84.685314,399.0,250.0,7,0-4
882,20090312433251036,1,0.003633,10,0,0,-394.991405,84.685314,399.0,250.0,7,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...
98182,20100018421509572,1,0.007850,10,0,0,-16.991405,8.685314,777.0,326.0,7,0-4
98183,20100018421509572,2,0.017000,10,0,0,-16.991405,8.685314,777.0,326.0,7,0-4
99163,20100019111032050,0,0.000000,1,0,0,-357.991405,-220.314686,436.0,555.0,7,0-4
99164,20100019111032050,1,0.004450,10,0,0,-360.991405,-219.314686,433.0,554.0,7,0-4


In [15]:
def feature_eng(df_sessions):
    df_final = pd.DataFrame()
    df_final['session_id'] = df_sessions['session_id'].unique()
    df_final['year'] = df_final['session_id'].apply(lambda x: int(str(x)[:2])).astype(np.uint8)
    df_final['month'] = df_final['session_id'].apply(lambda x: int(str(x)[2:4]) + 1).astype(np.uint8)
    df_final['weekday'] = df_final['session_id'].apply(lambda x: int(str(x)[4:6])).astype(np.uint8)
    df_final['hour'] = df_final['session_id'].apply(lambda x: int(str(x)[6:8])).astype(np.uint8)
    df_final['minute'] = df_final['session_id'].apply(lambda x: int(str(x)[8:10])).astype(np.uint8)
    df_final['second'] = df_final['session_id'].apply(lambda x: int(str(x)[10:12])).astype(np.uint8)
    df_final['ms'] = df_final['session_id'].apply(lambda x: int(str(x)[12:15])).astype(np.uint16)
    df_final['noise'] = df_final['session_id'].apply(lambda x: int(str(x)[15:17])).astype(np.uint8)
    return df_final

In [16]:
df_merge = pd.merge(data, target, on = 'session_id')
df_final = convert_time(df_merge)
df_final = df_final.set_index(['session_id'])
df_final.index.get_level_values('session_id')
day_map = {0: 'monday', 1: 'tuesday', 2:'wednesday', 3:'thursday', 4:'friday', 5:'saturday', 6:'sunday'}
df_final = pd.merge(df_merge, df_final, on= 'session_id')
df_final.weekday = df_final.weekday.map(day_map)
df_final.head()

NameError: name 'convert_time' is not defined

In [ ]:
"""
# Split the data into train, validation, and test sets
X_train, X_val, y_train, y_val = train_test_split(data, target, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)

# Create an SVM classifier with a linear kernel
svm = SVC(kernel='linear')

# Train the model on the training set
svm.fit(X_train, y_train)

# Predict on the validation set and evaluate the performance
y_pred = svm.predict(X_val)
print(classification_report(y_val, y_pred))

# Predict on the test set and evaluate the performance
y_pred_test = svm.predict(X_test)
print(classification_report(y_test, y_pred_test))
"""